In [86]:
import collections
import os
import json
import logging

from scipy.stats import entropy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import networkx as nx

if os.getcwd().endswith('notebook'):
    os.chdir('..')

from rna_learn.alphabet import ALPHABET_DNA, CODON_REDUNDANCY

In [87]:
sns.set(palette='colorblind', font_scale=1.3)
palette = sns.color_palette()
logging.basicConfig(level=logging.INFO, format="%(asctime)s (%(levelname)s) %(message)s")
logger = logging.getLogger(__name__)

In [88]:
db_path = os.path.join(os.getcwd(), 'data/db/seq.db')
engine = create_engine(f'sqlite+pysqlite:///{db_path}')

## Load codon bias

In [89]:
temperature_query = """
select s.assembly_accession, t.species_taxid, t.growth_tmp from assembly_source as s
left join (
    select species_taxid, growth_tmp from species_traits
) as t
on s.species_taxid = t.species_taxid
"""
temperatures_df = pd.read_sql(temperature_query, engine)
species_temperatures_df = temperatures_df[
    ['species_taxid', 'growth_tmp']].groupby('species_taxid').first().reset_index()

In [90]:
species_codon_ratios_path = os.path.join(os.getcwd(), 'data/species_codon_ratios.csv')

q = """
select 
    genome_size, species_taxid, species, genus, family, "order", class, phylum, superkingdom 
from species_traits
"""

species_codon_df = pd.merge(
    pd.read_csv(species_codon_ratios_path),
    species_temperatures_df,
    how='inner',
    on='species_taxid',
)
species_codon_df = pd.merge(
    species_codon_df,
    pd.read_sql(q, engine),
    how='inner',
    on='species_taxid',
)
species_codon_df.head()

,species_taxid,in_test_set,AAA_ratio,AAG_ratio,AAT_ratio,AAC_ratio,ACT_ratio,ACC_ratio,ACA_ratio,ACG_ratio,...,TTT_ratio,growth_tmp,genome_size,species,genus,family,order,class,phylum,superkingdom
0,7,True,0.082550,0.917450,0.327724,0.672276,0.024511,0.575353,0.036262,0.363875,...,0.102820,30.00,5369771.500,Azorhizobium caulinodans,Azorhizobium,Xanthobacteraceae,Rhizobiales,Alphaproteobacteria,Proteobacteria,Bacteria
1,9,False,0.919627,0.080373,0.858168,0.141832,0.454569,0.050026,0.446018,0.049387,...,0.919776,24.00,601699.243,Buchnera aphidicola,Buchnera,Erwiniaceae,Enterobacterales,Gammaproteobacteria,Proteobacteria,Bacteria
2,11,True,0.007302,0.992698,0.010730,0.989270,0.008586,0.425527,0.015789,0.550098,...,0.005406,26.00,3526440.800,Cellulomonas gilvus,Cellulomonas,Cellulomonadaceae,Micrococcales,Actinobacteria,Actinobacteria,Bacteria
3,14,True,0.664866,0.335134,0.775571,0.224429,0.458203,0.188052,0.304183,0.049563,...,0.810559,74.15,1959987.600,Dictyoglomus thermophilum,Dictyoglomus,Dictyoglomaceae,Dictyoglomales,Dictyoglomia,Dictyoglomi,Bacteria
4,19,True,0.551810,0.448190,0.440559,0.559441,0.099134,0.591478,0.097796,0.211592,...,0.497451,30.00,3722544.667,Pelobacter carbinolicus,Pelobacter,Desulfuromonadaceae,Desulfuromonadales,Deltaproteobacteria,Proteobacteria,Bacteria


In [91]:
traits_df = pd.read_sql('select * from species_traits', engine)
traits_df.columns

Index(['species_taxid', 'species', 'genus', 'family', 'order', 'class',
       'phylum', 'superkingdom', 'gram_stain', 'metabolism', 'pathways',
       'carbon_substrates', 'sporulation', 'motility', 'range_tmp',
       'range_salinity', 'cell_shape', 'isolation_source', 'd1_lo', 'd1_up',
       'd2_lo', 'd2_up', 'doubling_h', 'genome_size', 'gc_content',
       'coding_genes', 'optimum_tmp', 'optimum_ph', 'growth_tmp',
       'rRNA16S_genes', 'tRNA_genes', 'gram_stain.count', 'metabolism.count',
       'pathways.count', 'carbon_substrates.count', 'sporulation.count',
       'motility.count', 'range_tmp.count', 'range_salinity.count',
       'cell_shape.count', 'isolation_source.count', 'gram_stain.prop',
       'metabolism.prop', 'pathways.prop', 'carbon_substrates.prop',
       'sporulation.prop', 'motility.prop', 'range_tmp.prop',
       'range_salinity.prop', 'cell_shape.prop', 'isolation_source.prop',
       'd1_lo.count', 'd1_up.count', 'd2_lo.count', 'd2_up.count',
       'doubl

In [92]:
traits_df[[
    'gram_stain', 'metabolism', 'pathways',
    'carbon_substrates', 'sporulation', 'motility', 
    'range_salinity', 'cell_shape', 'isolation_source', 
    'doubling_h', 'optimum_ph', 'd1_lo', 'd1_up',
    'd2_lo', 'd2_up'
]].head()

,gram_stain,metabolism,pathways,carbon_substrates,sporulation,motility,range_salinity,cell_shape,isolation_source,doubling_h,optimum_ph,d1_lo,d1_up,d2_lo,d2_up
0,negative,aerobic,None,None,no,None,None,None,host_plant,NaN,NaN,NaN,NaN,NaN,NaN
1,negative,None,None,None,no,None,None,coccobacillus,host_animal_ectotherm,35.40,NaN,3.0,NaN,NaN,NaN
2,positive,facultative,cellulose_degradation,None,no,yes,None,bacillus,host_animal_endotherm,NaN,NaN,NaN,NaN,NaN,NaN
3,negative,anaerobic,cellulose_degradation,None,no,no,None,bacillus,water_hotspring,2.47,NaN,0.4,0.6,5.0,20.0
4,negative,anaerobic,"sulfur_reduction, iron_reduction, fermentation",None,no,None,None,bacillus,petroleum,NaN,NaN,0.5,0.7,1.2,3.0


In [114]:
traits_df[['range_salinity', 'species_taxid']].groupby('range_salinity').count()

,species_taxid
range_salinity,
extreme-halophilic,17
halophilic,1
halotolerant,3
moderate-halophilic,125
non-halophilic,160
stenohaline,1


In [115]:
traits_df['tRNA_genes'].describe()

count    2007.000000
mean       58.387014
std        19.089969
min        17.000000
25%        46.000000
50%        53.000000
75%        66.500000
max       177.667000
Name: tRNA_genes, dtype: float64

## Regression

In [82]:
ratio_columns = [c for c in species_codon_df.columns if c.endswith('_ratio')]

trait_name = 'genome_size'
df = species_codon_df[species_codon_df[trait_name].notnull()].reset_index(drop=True)

x_train = df[~df['in_test_set']][ratio_columns].values
y_train = df[~df['in_test_set']][trait_name].values

x_test = df[df['in_test_set']][ratio_columns].values
y_test = df[df['in_test_set']][trait_name].values

print(len(x_train), len(x_test))

1953 530


In [83]:
def fit_linear_regression(y_actual, x):
    X = sm.add_constant(x)
    model = sm.OLS(y_actual, X)
    results = model.fit()
    y_fit = results.predict(X)
    return results, y_fit

In [84]:
results, _ = fit_linear_regression(y_train, x_train)

In [85]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     47.77
Date:                Thu, 12 Nov 2020   Prob (F-statistic):          3.11e-267
Time:                        14:43:43   Log-Likelihood:                -30355.
No. Observations:                1953   AIC:                         6.080e+04
Df Residuals:                    1909   BIC:                         6.104e+04
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.365e+05   2.28e+04     19.162      0.000    3.92e+05    4.81e+05
x1         -2.434e+05   2.71e+05     -0.899      0.369   -7.75e+05    2.88e+05
x2          6.799e+05   2.69e+05      2.532      0.011    1.53e+05    1.21e+06
x3         -9.194e+04   4.68e+05     -0.196      0.844   -1.01e+06    8.27e+05
x4          5.284e+05   4.66e+05      1.133      0.257   -3.86e+05    1.44e+06
x5         -1.066e+06   8.15e+05     -1.308      0.191   -2.66e+06    5.32e+05
x6          5.671e+05   6.28e+05      0.903      0.366   -6.64e+05     1.8e+06
x7          1.951e+06   6.63e+05      2.943      0.003    6.51e+05    3.25e+06
x8         -1.016e+06   6.57e+05     -1.546      0.122    -2.3e+06    2.73e+05
x9         -2.149e+06   7.05e+05     -3.050      0.002   -3.53e+06   -7.67e+05
x10        -2.894e+06   6.37e+05     -4.544      0.000   -4.14e+06   -1.65e+06
x11          2.77e+06   1.03e+06      2.682      0.007    7.44e+05     4.8e+06
x12         4.291e+06   6.38e+05      6.728      0.000    3.04e+06    5.54e+06
x13        -1.479e+06   4.74e+05     -3.119      0.002   -2.41e+06   -5.49e+05
x14        -1.015e+05   5.87e+05     -0.173      0.863   -1.25e+06    1.05e+06
x15        -2.263e+06      1e+06     -2.258      0.024   -4.23e+06   -2.97e+05
x16        -5.005e+05   7.79e+05     -0.642      0.521   -2.03e+06    1.03e+06
x17        -1.192e+06   1.01e+06     -1.177      0.239   -3.18e+06    7.95e+05
x18         1.458e+05   8.75e+05      0.167      0.868   -1.57e+06    1.86e+06
x19         4.582e+06   1.27e+06      3.598      0.000    2.08e+06    7.08e+06
x20        -3.357e+05   9.07e+05     -0.370      0.711   -2.11e+06    1.44e+06
x21         9.352e+05   4.24e+05      2.205      0.028    1.03e+05    1.77e+06
x22          1.41e+06   4.39e+05      3.211      0.001    5.49e+05    2.27e+06
x23        -1.909e+06    4.2e+05     -4.549      0.000   -2.73e+06   -1.09e+06
x24          6.22e+05   2.88e+05      2.157      0.031    5.65e+04    1.19e+06
x25        -1.856e+05   2.89e+05     -0.642      0.521   -7.52e+05    3.81e+05
x26        -1.123e+06   3.75e+05     -2.997      0.003   -1.86e+06   -3.88e+05
x27         1.559e+06   3.74e+05      4.164      0.000    8.25e+05    2.29e+06
x28         6.659e+06   7.74e+05      8.605      0.000    5.14e+06    8.18e+06
x29        -3.454e+06   6.65e+05     -5.193      0.000   -4.76e+06   -2.15e+06
x30         3.458e+05   5.45e+05      0.635      0.526   -7.23e+05    1.41e+06
x31        -3.115e+06   4.92e+05     -6.335      0.000   -4.08e+06   -2.15e+06
x32        -3.484e+06    7.6e+05     -4.581      0.000   -4.98e+06   -1.99e+06
x33         2.618e+06   7.19e+05      3.643      0.000    1.21e+06    4.03e+06
x34        -1.856e+05   1.29e+06     -0.144      0.885   -2.71e+06    2.34e+06
x35         4.012e+06    6.1e+05      6.574      0.000    2.82e+06    5.21e+06
x3